In [2]:
# !conda create -y --name bedrock-router-eval python=3.11.8
# !conda init && activate bedrock-router-eval
# !conda install -n bedrock-router-eval ipykernel --update-deps --force-reinstall -y

In [11]:
!pip install -r requirements.txt

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
  Using cached pillow-10.4.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 72.4 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 118.6 MB/s eta 0:00:00
Using cached kiwisolver-1.4.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.4 MB)
Using cached pillow-10.4.0-cp311-cp311-manylinux_2_28_x86_64.whl (4.5 MB)
Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)


In [8]:
# Import necessary libraries
import boto3
import json
from dotenv import load_dotenv, find_dotenv
import os


# loading environment variables that are stored in local file dev.env
local_env_filename = 'bedrock-router-eval.env'
load_dotenv(find_dotenv(local_env_filename),override=True)

os.environ['REGION'] = os.getenv('REGION')
REGION = os.environ['REGION']

In [5]:
# Step 1: Define categories and difficulties
topics = ['code', 'summarization', 'general']
difficulty_levels = ['easy', 'medium', 'hard']
synthetic_data_model = "anthropic.claude-3-haiku-20240307-v1:0"

In [9]:
brt = boto3.client(service_name='bedrock-runtime', region_name=REGION)
data_gen_prompt = f"generate a json file with example prompts across the following topics {topics} and following category level {difficulty_levels}. Your answer will only contain the json formatted answer."
from botocore.exceptions import ClientError

def syntheticdatagen(topics,difficulty_levels):
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": data_gen_prompt}],
            }
        ],
    }
    # Convert the native request to JSON.
    request = json.dumps(native_request)
    
    try:
        # Invoke the model with the request.
        response = brt.invoke_model(modelId=synthetic_data_model, body=request)
    
    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{synthetic_data_model}'. Reason: {e}")
        exit(1)
    
    # Decode the response body.
    model_response = json.loads(response["body"].read())
    
    # Extract and print the response text.
    response_text = model_response["content"][0]["text"]
    print(data_gen_prompt)
    return response_text

In [10]:
syntheticdatagen(topics,difficulty_levels)

generate a json file with example prompts across the following topics ['code', 'summarization', 'general'] and following category level ['easy', 'medium', 'hard']. Your answer will only contain the json formatted answer.


'{\n  "code": {\n    "easy": [\n      "Write a simple function in Python that takes two numbers as input and returns their sum.",\n      "Create a JavaScript program that logs \'Hello, World!\' to the console.",\n      "Implement a basic sorting algorithm, such as bubble sort, in Java."\n    ],\n    "medium": [\n      "Develop a Python script that reads a CSV file, performs data cleaning, and outputs a cleaned dataset.",\n      "Write a JavaScript function that takes an array of numbers and returns the average of the array.",\n      "Create a Java program that implements a simple calculator with addition, subtraction, multiplication, and division operations."\n    ],\n    "hard": [\n      "Design and implement a RESTful API in Python using a web framework like Flask or Django.",\n      "Develop a JavaScript application that fetches data from an API and displays it on a web page using React or Angular.",\n      "Create a Java program that simulates a simple game of chess, including the 